In [2]:
cd /content/drive/My Drive/Colab Notebooks/dw-matrix2

/content/drive/My Drive/Colab Notebooks/dw-matrix2


In [3]:
!pip uninstall xgboost -y
!pip install xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl

Uninstalling xgboost-0.90:
  Successfully uninstalled xgboost-0.90
Processing ./xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl


In [4]:
!pip install eli5
!pip install -U tables
!pip install hyperopt

     |████████████████████████████████| 112kB 4.8MB/s 
     |████████████████████████████████| 4.3MB 4.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [5]:
import pandas as pd
import numpy as np

# from sklearn.dummy import DummyRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mean_absolute_error
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw-matrix2"

/content/drive/My Drive/Colab Notebooks/dw-matrix2


In [7]:
ls

data/               matrix2-day4.ipynb
LICENSE             README.md
matrix2-day2.ipynb  xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl
matrix2-day3.ipynb


In [8]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [0]:
org_df = df.copy()

In [0]:
df = org_df.copy()
bool_eur = df.price_currency == "EUR"
df.loc[bool_eur,'price_value'] *= 4.29 
df.loc[bool_eur,'price_currency'] = "PLN"

In [0]:
SUFFIX_CAT = "__cat"
for feat in df.columns:
    if isinstance(df[feat][0], list): continue
    
    factorized_values = df[feat].factorize(sort=True)[0]
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat+SUFFIX_CAT] = factorized_values

In [12]:
cat_feats = [x for x in df.columns if "__cat" in x]
cat_feats = [x for x in cat_feats if "price" not in x]
cat_feats

['created_at__cat',
 'seller_address__cat',
 'seller_name__cat',
 'seller_type__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_poduszka-powietrzna-chroniąca-kolana__cat',
 'feature_kurtyny-powietrzne__cat',
 'feature_klimatyzacja-dwustrefowa__cat',
 'feature_światła-led__cat',
 'feature_czujnik-zmierzchu__cat',
 'feature_elektrycznie-ustawiane-lusterka__cat',
 'feature_asr-(kontrola-trakcji)__cat',
 'feature_poduszka-powietrzna-kierowcy__cat',
 'feature_cd__cat',
 'feature_elektryczne-szyby-przednie__cat',
 'feature_poduszka-powietrzna-pasażera__cat',
 'feature_system-start-stop__cat',
 'feature_światła-do-jazdy-dziennej__cat',
 'feature_komputer-pokładowy__cat',
 'feature_elektryczne-szyby-tylne__cat',
 'feature_klimatyzacja-manualna__cat',
 'feature_tapicerka-welurowa__cat',
 'feature_czujnik-deszczu__cat',
 'feature_światła-przeciwmgielne__cat',
 'feature_ogrzewanie-postojowe__cat',
 'feature_radio-niefabryczne__cat',
 'feature_regulowane-zawieszenie__cat',
 'feature_

In [13]:
X = df[cat_feats].values
y = df["price_value"].values

xgb_params = {
    "max_depth":10,
    'n_estimators':100,
    "learning_rate":0.1,
    "seed":0,
    "tree_method":"gpu_hist"
}

m = xgb.XGBRegressor(**xgb_params)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli = eli5.show_weights(imp, feature_names=cat_feats, top=50)
eli

Weight,Feature
0.7433 ± 0.0208,param_rok-produkcji__cat
0.2212 ± 0.0026,param_moc__cat
0.2022 ± 0.0048,param_pojemność-skokowa__cat
0.1336 ± 0.0061,param_skrzynia-biegów__cat
0.0354 ± 0.0019,param_marka-pojazdu__cat
0.0291 ± 0.0024,param_model-pojazdu__cat
0.0281 ± 0.0014,param_metalik__cat
0.0164 ± 0.0007,param_napęd__cat
0.0157 ± 0.0005,param_typ__cat
0.0142 ± 0.0007,param_kod-silnika__cat


In [14]:
temp = pd.DataFrame(zip(cat_feats, imp.feature_importances_), columns=["Feature","Perm_Importance"]).sort_values(by="Perm_Importance", ascending=False)
important_features_xgb = list(temp["Feature"][:40])
important_features_xgb.remove("car_id__cat")
important_features_xgb.remove("seller_address__cat")
important_features_xgb

['param_rok-produkcji__cat',
 'param_moc__cat',
 'param_pojemność-skokowa__cat',
 'param_skrzynia-biegów__cat',
 'param_marka-pojazdu__cat',
 'param_model-pojazdu__cat',
 'param_metalik__cat',
 'param_napęd__cat',
 'param_typ__cat',
 'param_kod-silnika__cat',
 'seller_name__cat',
 'param_wersja__cat',
 'param_przebieg__cat',
 'param_bezwypadkowy__cat',
 'created_at__cat',
 'param_rodzaj-paliwa__cat',
 'param_uszkodzony__cat',
 'param_liczba-miejsc__cat',
 'param_stan__cat',
 'feature_kamera-cofania__cat',
 'param_faktura-vat__cat',
 'feature_elektrycznie-ustawiane-fotele__cat',
 'feature_regulowane-zawieszenie__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_światła-xenonowe__cat',
 'param_kolor__cat',
 'param_kraj-pochodzenia__cat',
 'param_liczba-drzwi__cat',
 'feature_tapicerka-skórzana__cat',
 'feature_łopatki-zmiany-biegów__cat',
 'feature_nawigacja-gps__cat',
 'feature_światła-led__cat',
 'feature_czujnik-martwego-pola__cat',
 'param_vin__cat',
 'feature_wspomaganie

In [0]:
def run_model(model, feats):
    X = df[feats].values
    y = df["price_value"].values
    model = model
    scores = -cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error", n_jobs=-1)
    print(f"n {n} lr {lr} max_depth {max_d}: \t score: {round(np.mean(scores))}\t std: {round(np.std(scores))}\t score+2std: {round(np.mean(scores) + 3*np.std(scores))}")

In [25]:
max_d = 10
n = 100
lr = 0.1

xgb_params = {
        "max_depth":max_d,
        'n_estimators':n,
        "learning_rate":lr,
        "seed":0,
        "tree_method":"gpu_hist",
        # "objective":"reg:squarederror",
        }

run_model(xgb.XGBRegressor(**xgb_params), important_features_xgb)

n 100 lr 0.1 max_depth 10: 	 score: 7151.0	 std: 56.0	 score+2std: 7320.0


In [0]:
def run_model_ho(model, feats=important_features_xgb):
    X = df[feats].values
    y = df["price_value"].values
    model = model
    scores = -cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error", n_jobs=-1)
    # print(f"n {n} lr {lr} max_depth {max_d}: \t score: {round(np.mean(scores))}\t std: {round(np.std(scores))}\t score+std: {round(np.mean(scores) + np.std(scores))}")
    return round(np.mean(scores),1), round(np.std(scores),1)

In [31]:
def obj_func(params):
    print("Training with params: ")
    print(params)

    mean_mae, score_std = run_model_ho(xgb.XGBRegressor(**params), feats=important_features_xgb)

    return {"loss": np.abs(mean_mae), "status": STATUS_OK}

# space
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate', np.arange(0.05,0.31, 0.05,)),
    'max_depth':        hp.choice('max_depth', np.arange(7, 12, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.1),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.1),
    "tree_method":      "gpu_hist",
    'n_estimators':     100,
    'seed':             0
}

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=50)
best

Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 100, 'seed': 0, 'subsample': 0.8, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'seed': 0, 'subsample': 0.5, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 100, 'seed': 0, 'subsample': 0.5, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 100, 'seed': 0, 'subsample': 0.6000000000000001, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 100, 'seed': 0, 'subsample': 0.9, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'seed': 0

{'colsample_bytree': 0.8, 'learning_rate': 1, 'max_depth': 4, 'subsample': 1.0}